In [1]:
%pip install -qq langchain==0.1.10
%pip install -qq langgraph==0.0.26
%pip install -qq pydantic==1.10.8
%pip install -qq langchainhub==0.1.14
# %pip install -qq transformers==4.38.2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Setup Ollama
Installing `lspci` with apt
```bash
$ sudo apt-get update
$ sudo apt-get install pciutils
```

```bash
$ curl -fsSL https://ollama.com/install.sh | sh
```
```bash
$ ollama serve
```
```bash
$ ollama pull openhermes
$ ollama pull nexusraven
```

### Defining the Agent State and Tools

In [1]:
from datetime import datetime

from langchain_core.tools import tool
from langgraph.prebuilt import ToolExecutor


@tool
def get_now(format: str = "%Y-%m-%d %H:%M:%S"):
    """Get the current time"""
    return datetime.now().strftime(format)


tools = [get_now]
tool_executor = ToolExecutor(tools)

### Integrating Ollama and Defining the Workflow

In [2]:
import operator
from typing import Annotated, TypedDict, Union

from langchain import hub
from langchain.agents import create_react_agent
from langchain_community.chat_models import ChatOllama
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]


model = ChatOllama(model="openhermes")
prompt = hub.pull("hwchase17/react")

agent_runnable = create_react_agent(model, tools, prompt)

In [3]:
from IPython.display import display, Markdown, HTML


def log_markdown(message_markdown: str):
    display(Markdown(message_markdown))


def log_info(message: str):
    display(HTML(f'<span style="color: green;">Info: {message}</span>'))

In [4]:
from langgraph.prebuilt import ToolInvocation


def execute_tools(state):
    log_markdown("**Called execute tools**")
    messages = [state["agent_outcome"]]
    last_message = messages[-1]

    tool_name = last_message.tool
    log_markdown(f"Calling Tool: `{tool_name}`")

    action = ToolInvocation(
        tool=tool_name,
        tool_input=last_message.tool_input,
    )

    response = tool_executor.invoke(action)
    return {"intermediate_steps": [(state["agent_outcome"], response)]}


def run_agent(state):
    """
    #if you want to better manages intermediate steps
    inputs = state.copy()
    if len(inputs['intermediate_steps']) > 5:
        inputs['intermediate_steps'] = inputs['intermediate_steps'][-5:]
    """
    agent_outcome = agent_runnable.invoke(state)
    return {"agent_outcome": agent_outcome}


def should_continue(state):
    messages = [state["agent_outcome"]]
    last_message = messages[-1]
    if "Action" not in last_message.log:
        return "end"
    else:
        return "continue"

In [5]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)


workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent", should_continue, {"continue": "action", "end": END}
)


workflow.add_edge("action", "agent")
app = workflow.compile()

input_text = "Whats the current time?"

In [10]:
inputs = {"input": input_text, "chat_history": []}
results = []
for s in app.stream(inputs):
    result = list(s.values())[0]
    results.append(result)
    print(result)

{'agent_outcome': AgentAction(tool='get_now', tool_input='No input required for this action.', log='To find out the current time, we can use the get_now tool.\nAction: get_now\nAction Input: No input required for this action.')}


**Called execute tools**

Calling Tool: `get_now`

{'intermediate_steps': [(AgentAction(tool='get_now', tool_input='No input required for this action.', log='To find out the current time, we can use the get_now tool.\nAction: get_now\nAction Input: No input required for this action.'), 'No input required for this action.')]}
{'agent_outcome': AgentFinish(return_values={'output': '<current_time>'}, log="The current time is the final answer to the original input question. The time will be displayed in the format specified by '%Y-%m-%d %H:%M:%S'.\n\nFinal Answer: <current_time>")}
{'input': 'Whats the current time?', 'chat_history': [], 'agent_outcome': AgentFinish(return_values={'output': '<current_time>'}, log="The current time is the final answer to the original input question. The time will be displayed in the format specified by '%Y-%m-%d %H:%M:%S'.\n\nFinal Answer: <current_time>"), 'intermediate_steps': [(AgentAction(tool='get_now', tool_input='No input required for this action.', log='To find out the current time, we can use the g